### Convert the tlp file output by GrouseFlocks to MyJSON 
* We treat each open metanode as a "true metanode" that has underlying leaf nodes, but the closed metanode does not have underlying leaf nodes.  They are both in the data structure "metanodes"

In [157]:
from tulip import tlp
from shapely.geometry import *
from tlp2myjson import chop_segment
import os
import math
import json
from pprint import pprint
SQUARE_ROOT_2 = math.sqrt(2)

In [158]:
DATA_DIR = '../../../data/real-world-compiled/partition/'
FILENAME = 'partition-bcsstk33-grouseflocks-open-8'
grousetlp_filepath = os.path.join(DATA_DIR, FILENAME + '.tlp')
myjson_filepath = os.path.join(DATA_DIR, FILENAME + '.json')

In [159]:
graph = tlp.loadGraph(grousetlp_filepath)
hierarchy_subgraphs = graph.getSubGraph('1__HierarchySubGraphs')
# hierarchy_unnamed = hierarchy_subgraphs.getSubGraph('unnamed')
final_layout = graph.getSubGraph('1__Final Layout Display')
assert(hierarchy_subgraphs is not None)
# assert(hierarchy_unnamed)
assert(final_layout is not None)
view_meta_graph = graph.getGraphProperty('viewMetaGraph')

In [160]:
metanodes = {}
leaf_nodes = []
edges = []
max_level = 1
root = [x.id for x in graph.getSubGraph('1__Hierarchy').getNodes()][0]
# a mapping from lable "Contracted xx" in "Final Layout Display" to metanode id
label2metanode_id = {}
disp_id2metanode_id = {}
parents = {}
print('root = ', root)

root =  8738


In [161]:
view_label = final_layout.getStringProperty('viewLabel')
view_disp2hier = final_layout.getIntegerProperty('view Display Node to Hier')
view_hier2disp = final_layout.getIntegerProperty('view Hier Node to Display')
view_layout = final_layout.getLayoutProperty('viewLayout')
view_size = final_layout.getSizeProperty('viewSize')

for n in final_layout.getNodes():
    center_coor = view_layout[n]
    width = view_size[n][0]
    if view_hier2disp[n] == -1:
        metanode_id = view_disp2hier[n]
        label2metanode_id[view_label[n]] = metanode_id
        disp_id2metanode_id[n.id] = metanode_id
        metanodes[metanode_id] = {
            'id': metanode_id,   
            'parent_metanode': root,  
            # FIXME this might not be true for multi-level graph
            # instead find the metanode_id in the "1_HierarchySubGraphs"
            'desc_metanodes': {},
            'level': 0,
            'leaf_nodes': {},
            'diameter': width,
            'geometry': Point(center_coor.x(), center_coor.y()).buffer(width / 2.0, cap_style=CAP_STYLE.round)
        }
# pprint(label2metanode_id)
# pprint(disp_id2metanode_id)

In [162]:
# Recursively find all open metanodes from top to bottom

def dfs(g, cur_metanode_id, depth):
    print('Visiting graph {}, current metanode id: {}, depth: {}'.format(g, cur_metanode_id, depth))
    view_hier2disp = g.getIntegerProperty('view Hier Node to Display')
    view_disp2hier = g.getIntegerProperty('view Display Node to Hier')
    view_layout = g.getLayoutProperty('viewLayout')
    view_size = g.getSizeProperty('viewSize')
                
    for n in g.getNodes():
        if view_hier2disp[n] == -1:
            # open metanode
            # Do not add the root (whole graph)
            if depth > 0:
                # record position and size
                center_coor = view_layout[n]
                d = view_size[n][0] * SQUARE_ROOT_2
                metanodes[n.id] = {
                    'id': n.id,
                    'parent_metanode': cur_metanode_id,
                    'desc_metanodes': {},
                    'level': depth,
                    'leaf_nodes': {},
                    'geometry': Point(center_coor.x(), center_coor.y()).buffer(d / 2.0, cap_style=CAP_STYLE.round),
                    'diameter': d
                }
                print('Open metanode:', metanodes[n.id])
            
            corr_g = view_meta_graph[n]
            assert(corr_g is not None)
            dfs(corr_g, n.id, depth + 1)
        elif view_disp2hier[n] == -1:
            # close metanode.  Find out its metanode id using the label
            corr_g = view_meta_graph[n]
            assert(corr_g is not None)
            true_metanode_id = label2metanode_id[corr_g.getName()]
            
            # Add it to the current metanode
            if cur_metanode_id is not None:
                print('Adding metanode {} to metanode {}'.format(true_metanode_id, cur_metanode_id))
                # metanodes[cur_metanode_id]['desc_metanodes'][true_metanode_id] = True
                metanodes[true_metanode_id]['parent_metanode'] = cur_metanode_id
        else:
            # Leaf node
            if cur_metanode_id is not None:
                # The id of this node is just the display id
                print('Adding leaf node {} to metanode {}'.format(n.id, cur_metanode_id))
                metanodes[cur_metanode_id]['leaf_nodes'][n.id] = True
                parents[n.id] = cur_metanode_id

In [163]:
dfs(graph.getSubGraph('1__Hierarchy'), None, 0)

Visiting graph <graph "1__Hierarchy" (id 3) >, current metanode id: None, depth: 0
Visiting graph <graph "unnamed" (id 5) >, current metanode id: 8738, depth: 1
Adding metanode 8740 to metanode 8738
Adding metanode 8741 to metanode 8738
Adding metanode 8742 to metanode 8738
Adding metanode 8743 to metanode 8738
Adding metanode 8744 to metanode 8738
Adding metanode 8745 to metanode 8738
Adding metanode 8746 to metanode 8738
Adding metanode 8747 to metanode 8738
Adding metanode 8748 to metanode 8738
Adding metanode 8749 to metanode 8738
Adding metanode 8750 to metanode 8738
Adding metanode 8751 to metanode 8738
Adding metanode 8752 to metanode 8738
Adding metanode 8753 to metanode 8738
Adding metanode 8754 to metanode 8738
Adding metanode 8755 to metanode 8738
Adding metanode 8756 to metanode 8738
Adding metanode 8757 to metanode 8738
Adding metanode 8758 to metanode 8738
Adding metanode 8759 to metanode 8738
Adding metanode 8760 to metanode 8738
Adding metanode 8761 to metanode 8738
Add

Adding leaf node 4108 to metanode 8877
Adding leaf node 4884 to metanode 8877
Adding leaf node 4110 to metanode 8877
Adding leaf node 4499 to metanode 8877
Adding leaf node 4111 to metanode 8877
Adding leaf node 5372 to metanode 8877
Adding leaf node 4887 to metanode 8877
Adding leaf node 5373 to metanode 8877
Adding leaf node 5374 to metanode 8877
Adding leaf node 4114 to metanode 8877
Adding leaf node 4890 to metanode 8877
Adding leaf node 4411 to metanode 8877
Adding leaf node 4412 to metanode 8877
Adding leaf node 4416 to metanode 8877
Adding leaf node 4421 to metanode 8877
Adding leaf node 4131 to metanode 8877
Adding leaf node 4034 to metanode 8877
Adding leaf node 4132 to metanode 8877
Adding leaf node 4520 to metanode 8877
Adding leaf node 5781 to metanode 8877
Adding leaf node 4036 to metanode 8877
Adding leaf node 4910 to metanode 8877
Adding leaf node 4038 to metanode 8877
Adding leaf node 5784 to metanode 8877
Adding leaf node 4525 to metanode 8877
Adding leaf node 4914 to 

In [164]:
# Fill in the fields "desc_metanode" and "leaf_nodes" according to field "parent_metanode"
for metanode_id, mn in metanodes.items():
    p = mn['parent_metanode']
    if mn['level'] > max_level:
        max_level = mn['level']
    print('at ', metanode_id)
    while p is not None and p in metanodes:
        cur_mn = metanodes[p]
        cur_mn['desc_metanodes'][metanode_id] = True
        dict.update(cur_mn['leaf_nodes'], mn['leaf_nodes'])
        print('\t adding to ', p)
        p = cur_mn['parent_metanode']

at  8740
at  8741
at  8742
at  8743
at  8744
at  8745
at  8746
at  8747
at  8748
at  8749
at  8750
at  8751
at  8752
at  8753
at  8754
at  8755
at  8756
at  8757
at  8758
at  8759
at  8760
at  8761
at  8762
at  8763
at  8764
at  8765
at  8766
at  8767
at  8768
at  8769
at  8770
at  8771
at  8772
at  8773
at  8774
at  8775
at  8776
at  8777
at  8778
at  8779
at  8780
at  8781
at  8783
at  8784
at  8785
at  8786
at  8787
at  8788
at  8789
at  8791
at  8792
at  8793
at  8794
at  8795
at  8796
at  8797
at  8798
at  8799
at  8800
at  8801
at  8802
at  8803
at  8804
at  8805
at  8806
at  8807
at  8808
at  8810
at  8811
at  8812
at  8813
at  8814
at  8815
at  8816
at  8817
at  8818
at  8819
at  8820
at  8821
at  8822
at  8823
at  8824
at  8825
at  8826
at  8827
at  8828
at  8829
at  8830
at  8831
at  8832
at  8833
at  8834
at  8835
at  8836
at  8837
at  8838
at  8839
at  8840
at  8841
at  8842
at  8843
at  8844
at  8845
at  8846
at  8847
at  8848
at  8849
at  8850
at  8851
at  8852
at  8853
a

In [165]:
# Find out all leaf nodes
view_hier_attr = final_layout.getIntegerProperty('view Hier Node to Display')
sub_view_layout = final_layout.getLayoutProperty('viewLayout')
sub_view_size = final_layout.getSizeProperty('viewSize')
leaf_mapping = {}

for n in final_layout.getNodes():
    center_coor = sub_view_layout[n]
    width = sub_view_size[n][0]
    # print(n.id, center_coor, diameter)
    if view_hier_attr[n] != -1:
        # leaf nodes
        print('leaf: ', n.id)
        # Leaf node is only possible when a metanode is open
        assert(n.id in parents)      
        new_node = {
            'id': n.id,
            'parent_metanode': parents[n.id], 
            'geometry': Point(center_coor.x(), center_coor.y()).buffer(width / 2.0, cap_style=CAP_STYLE.square),
            'diameter': width * SQUARE_ROOT_2,
        }     
        leaf_mapping[n.id] = new_node
        leaf_nodes.append(new_node)

print(len(leaf_nodes))

leaf:  1538
leaf:  1539
leaf:  8463
leaf:  8468
leaf:  8469
leaf:  8472
leaf:  8476
leaf:  8477
leaf:  8479
leaf:  8480
leaf:  8481
leaf:  8482
leaf:  8483
leaf:  8486
leaf:  8487
leaf:  8488
leaf:  8490
leaf:  8491
leaf:  1572
leaf:  8495
leaf:  8499
leaf:  8500
leaf:  1580
leaf:  8502
leaf:  8503
leaf:  814
leaf:  815
leaf:  8507
leaf:  817
leaf:  8511
leaf:  8512
leaf:  824
leaf:  825
leaf:  826
leaf:  8517
leaf:  827
leaf:  8519
leaf:  829
leaf:  8521
leaf:  8522
leaf:  833
leaf:  5456
leaf:  5457
leaf:  847
leaf:  848
leaf:  850
leaf:  864
leaf:  8559
leaf:  5486
leaf:  5487
leaf:  5488
leaf:  5489
leaf:  5491
leaf:  882
leaf:  883
leaf:  5497
leaf:  886
leaf:  887
leaf:  890
leaf:  8598
leaf:  3984
leaf:  3985
leaf:  5525
leaf:  8603
leaf:  3989
leaf:  5528
leaf:  3990
leaf:  5529
leaf:  5530
leaf:  3994
leaf:  3995
leaf:  8634
leaf:  8635
leaf:  5565
leaf:  5566
leaf:  5567
leaf:  5568
leaf:  8645
leaf:  5569
leaf:  4032
leaf:  8647
leaf:  4034
leaf:  8649
leaf:  5573
leaf:  403

In [166]:
view_label = final_layout.getStringProperty('viewLabel')
view_layout = final_layout.getLayoutProperty('viewLayout')

for e in final_layout.getEdges():
    src, tgt = graph.ends(e)
    if src.id in leaf_mapping:
        src_id = src.id
        src_geo = leaf_mapping[src.id]['geometry']
    else:
        src_id = label2metanode_id[view_label[src]]
        src_geo = metanodes[src_id]['geometry']
        
    if tgt.id in leaf_mapping:
        tgt_id = tgt.id
        tgt_geo = leaf_mapping[tgt.id]['geometry']
    else:
        tgt_id = label2metanode_id[view_label[tgt]]
        tgt_geo = metanodes[tgt_id]['geometry']
        
    edge_id = '{}-{}'.format(src_id, tgt_id) 
    edges.append({
        'id': edge_id,
        'ends': (src_id, tgt_id),
        'geometry': chop_segment(src_geo, tgt_geo)
    })
print(len(edges))

21093


In [167]:
bbox = tlp.computeBoundingBox(final_layout)

In [168]:
# Use the mapping function from shapely to serialize the geometry objects
for n in leaf_nodes:
    n['geometry'] = mapping(n['geometry'])
for e in edges:
    e['geometry'] = mapping(e['geometry'])
for _, n in metanodes.items():
    n['geometry'] = mapping(n['geometry'])

json_data = {
    'leaf_nodes': leaf_nodes,
    'edges': edges,
    'height': max_level + 1,  # the max level of metanodes + one level of leaf nodes (the bottom)
    'root': root,
    'metanodes': metanodes,
    'bounding_box': [[bbox[0].x(), bbox[0].y()], [bbox[1].x(), bbox[1].y()]]
}

In [169]:
json.dump(json_data, open(myjson_filepath, 'w'), indent=2)
print('Converted to ', myjson_filepath, ' #nodes:', len(leaf_nodes), ' #edges: ', len(edges), 
      '#metanodes:', len(metanodes),
      ' height: ', json_data['height'])

Converted to  ../../../data/real-world-compiled/partition/partition-bcsstk33-grouseflocks-open-8.json  #nodes: 466  #edges:  21093 #metanodes: 200  height:  2
